In [ ]:
!pip install requests beautifulsoup4

In [1]:
!pip install PyPDF2
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install tiktoken
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install langchain-community langchain-openai faiss-cpu pypdf beautifulsoup4 requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 8.5 MB/s eta 0:00:00


In [3]:
import os
import requests
from bs4 import BeautifulSoup
import PyPDF2
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

def download_and_extract_ipc():
    #/content/IPC_186045_removed_removed.pdf
    pdf_path = "/content/IPC_186045_removed_removed.pdf"
    if not os.path.exists(pdf_path):
        url = "https://www.indiacode.nic.in/repealedfileopen?rfilename=A1860-45.pdf"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pdf_link = None

            for a in soup.find_all('a', href=True):
                if ".pdf" in a['href']:
                    pdf_link = a['href']
                    break

            if pdf_link:
                pdf_response = requests.get(pdf_link)
                with open(pdf_path, "wb") as file:
                    file.write(pdf_response.content)
                print("IPC Document Downloaded Successfully!")
            else:
                print("PDF link not found on the page.")
        else:
            print("Failed to access IPC webpage.")

    text = extract_text_from_pdf(pdf_path)
    with open("Indian_Penal_Code.txt", "w", encoding="utf-8") as text_file:
        text_file.write(text)
    print("Text extracted and saved successfully!")
    return text

def create_chatbot():
    os.environ["OPENAI_API_KEY"] = "yourapikey"  # Set your API key here

    text = download_and_extract_ipc()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_text(text)

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts, embeddings, normalize_L2=True)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo")
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

    print("Chatbot is ready! Type 'exit' to stop.")
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        response = qa_chain.invoke(query)
        print("Bot:")
        print(response['result'].replace('. ', '.'))
if __name__ == "__main__":
    create_chatbot()


Failed to access IPC webpage.


FileNotFoundError: [Errno 2] No such file or directory: '/content/IPC_186045_removed_removed.pdf'